# FP-Growth
FP-growth算法是一种基于Apriori构建的高效发现频繁集的算法。
> 优点：一般要快于Apriori<br>
> 缺点：实现比较困难，在某些数据集上性能会下降<br>
> 适用数据类型：标称型数据

FP-Growth的一般流程
> 1. 收集数据:使用任意方法。
> 2. 准备数据:由于存储的是集合,所以需要离散数据。如果要处理连续数据,需要将它们量化为离散值。
> 3. 分析数据:使用任意方法。
> 4. 训练算法:构建一个FP树,并对树进行挖据。
> 5. 测试算法:没有测试过程。
> 6. 使用算法:可用于识别经常出现的元素项,从而用于制定决策、推荐元素或进行预测

## 1. FP树的实现
### a. 创建树的数据结构
FP树要比其他树更加复杂需要创建一个类来保存树的每个节点。

In [74]:
class treeNode:
    def __init__(self, nameValue, numOccur, parentNode):
        self.name = nameValue
        self.count = numOccur
        self.nodeLink = None   # 是否有相似节点
        self.parent = parentNode
        self.children = {}
        
    def inc(self, numOccur):
        self.count += numOccur
        
    def disp(self, ind=1):
        # 将树以文本形式显示
        print("    " * ind, self.name, ' ', self.count)
        for child in self.children.values():
            child.disp(ind+1)

In [75]:
rootNode = treeNode('pyramid', 9, None)
rootNode.children['eye'] = treeNode('eye', 13, None)
rootNode.disp()

     pyramid   9
         eye   13


### b. 构建FP树
    除了FP树本身外，还要构建一个头指针表，利用头指针表可以快速访问给定类型的所有元素，头指针表中还保存了所有元素的总数。
    第一次遍历数据集会获得每个元素项出现的频率，去掉不满足最小支持度的项后就可以构建FP树了。在构建时，读入每个项集并将其添加到一条已存在的路径中，如果不存在该路径，那么则创建。

In [21]:
def createInitSet(dataSet):
    """
    生成frozenset，并统计所有项集的出现次数
    """
    retDict = {}
    for trans in dataSet:
        if frozenset(trans) not in retDict.keys():
            retDict[frozenset(trans)] = 1
        else:
            retDict[frozenset(trans)] += 1
    return retDict

def updateHeader(nodeToTest, targetNode):
    """
    更新头指针，建立相同元素之间的关系，例如左边的r指向右边的r
    
    Parameters
    -----------
        nodeToTest : 满足最小支持度所有元素
        targetNode : Tree对象的子节点
    """
    while nodeToTest.nodeLink is not None:
        nodeToTest = nodeToTest.nodeLink
    nodeToTest.nodeLink = targetNode
    
def updateTree(items, inTree, headerTable, count):
    """
    更新FP树
    
    Parameters
    -----------
        items :        满足最小支持度排序后的元素key的数组
        inTree :       空的Tree对象
        headerTable :  满足最小支持度的所有元素
        count :        原数据集中每一组key出现的次数
    """
    
    # items[0]为测试项集第一个元素，比如frozenset(['a', 'b', 'c'])，items[0]就是a
    # 如果该元素在 inTree.children 这个字典中，就进行累加
    # 如果该元素不存在 就 inTree.children 字典中新增key，value为初始化的 treeNode 对象
    if items[0] in inTree.children:
        # 叠加出现次数
        inTree.children[items[0]].inc(count)
    else:
        # 如果不存在子节点，为该inTree添加子节点
        inTree.children[items[0]] = treeNode(items[0], count, inTree)
        # 如果dist字典的值的第二个元素为None，那么就设置本节点为对应的树节点
        if headerTable[items[0]][1] is None:
            # headerTable只记录第一次节点出现的位置
            headerTable[items[0]][1] = inTree.children[items[0]]
        else:
            # 更新子树的nodeLink值
            updateHeader(headerTable[items[0]][1], inTree.children[items[0]])
    if len(items) > 1:
        updateTree(items[1:], inTree.children[items[0]], headerTable, count)
        
def createTree(dataSet, minSup=1):
    """
    生成FP树
    
    Parameters
    -----------
        dataSet : dist{行：出现次数}的样本数据 
        minSup :  最小支持度
    Returns
    -----------
        retTree : FP-tree
        headerTable :满足最小支持度的所有元素
    """
    
    headerTable = {}
    # trans是数据集中的每条记录
    for trans in dataSet:
        # 统计每行中所有元素出现的总次数
        for item in trans:
            # 例如：{'ababa': 3}，headerTable['a'] = 3+3+3 = 9, headerTable['b'] = 6
            headerTable[item] = headerTable.get(item, 0) + dataSet[trans]
    # 删除headerTable中支持度<minSup的元素
    for k in list(headerTable.keys()):
        if headerTable[k] < minSup:
            del(headerTable[k])
    
    # 所有频繁项的不重复集合
    freqItemSet = set(headerTable.keys())
    if len(freqItemSet) == 0:
        return None, None
    for k in headerTable:
        # 格式化，dist{元素key: [元素次数, None]}
        headerTable[k] = [headerTable[k], None]
    
    retTree = treeNode('Null Set', 1, None)
    for tranSet, count in dataSet.items():
        localD = {}
        for item in tranSet:
            if item in freqItemSet:
                # headerTable[item]为上面dist字典中的列表，所以localD[item]就是item的出现次数
#                 print('headerTable[item][0]=', headerTable[item][0], headerTable[item])
                localD[item] = headerTable[item][0]
#         print('localD=', localD)
        if len(localD) > 0:
            # p = key, value是字典的项，所以p[1]就是使用value从大到小排序
            # orderedItems表示取出原组的key值，也就是字母本身
            orderedItems = [v[0] for v in sorted(localD.items(), key=lambda p: p[1], reverse=True)]
#             print('orderedItems = ', orderedItems, 'headerTable', headerTable, '\n\n')
            # 填充树，通过有序的orderedItems的第一位进行顺序填充第一层的子节点
            updateTree(orderedItems, retTree, headerTable, count)
    return retTree, headerTable

In [12]:
def loadSimpDat():
    simpDat = [['r', 'z', 'h', 'j', 'p'],
               ['z', 'y', 'x', 'w', 'v', 'u', 't', 's'],
               ['z'],
               ['r', 'x', 'n', 'o', 's'],
            #    ['r', 'x', 'n', 'o', 's'],
               ['y', 'r', 'x', 'z', 'q', 't', 'p'],
               ['y', 'z', 'x', 'e', 'q', 's', 't', 'm']]
    return simpDat

In [13]:
simpDat = loadSimpDat()
initSet = createInitSet(simpDat)
initSet

{frozenset({'h', 'j', 'p', 'r', 'z'}): 1,
 frozenset({'s', 't', 'u', 'v', 'w', 'x', 'y', 'z'}): 1,
 frozenset({'z'}): 1,
 frozenset({'n', 'o', 'r', 's', 'x'}): 1,
 frozenset({'p', 'q', 'r', 't', 'x', 'y', 'z'}): 1,
 frozenset({'e', 'm', 'q', 's', 't', 'x', 'y', 'z'}): 1}

In [28]:
myFPtree, myHeaderTab = createTree(initSet, 3)
myFPtree.disp()
print(myHeaderTab)

     Null Set   1
         z   5
             r   1
             x   3
                 y   3
                     s   2
                         t   2
                     r   1
                         t   1
         x   1
             s   1
                 r   1
{'r': [3, <__main__.treeNode object at 0x7f1eec1f07f0>], 'z': [5, <__main__.treeNode object at 0x7f1eec1f0208>], 'y': [3, <__main__.treeNode object at 0x7f1eec1f0358>], 'x': [4, <__main__.treeNode object at 0x7f1eec1f0898>], 's': [3, <__main__.treeNode object at 0x7f1eec1f0198>], 't': [3, <__main__.treeNode object at 0x7f1eec1f0160>]}


## 2. 从FP树中挖掘频繁项集
基本步骤：
> 1. 从FP树中获得条件模式基
> 2. 利用条件模式基构建一个条件FP树
> 3. 迭代重复步骤1，2，直到树包含一个元素项为止

### a. 抽取条件模式基
从保存的headerTable的单个频繁元素开始，以该元素为结尾，生成前缀路径，就是该元素的条件模式基。具体参见《机器学习实战》p231

In [24]:
def ascendTree(leafNode, prefixPath):
    """
    从当前元素开始，所有可以返回到根节点的路径
    """
    if leafNode.parent != None:
        prefixPath.append(leafNode.name)
        ascendTree(leafNode.parent, prefixPath)
        
def findPrefixPath(basePat, treeNode):
    condPats = {}
    # 对树上节点的nodeLink值进行循环
    while treeNode != None:
        prefixPath = []
        # 寻找该节点的到根节点的路径
        ascendTree(treeNode, prefixPath)
        if (len(prefixPath)) > 1:
            # prefixPath[1:]剔除当前节点
            condPats[frozenset(prefixPath[1:])] = treeNode.count
        treeNode = treeNode.nodeLink
    return condPats

In [46]:
findPrefixPath('r', myHeaderTab['r'][1])

{frozenset({'z'}): 1, frozenset({'s', 'x'}): 1, frozenset({'x', 'y', 'z'}): 1}

### b. 创建条件FP树
使用上面的条件模式基作为输入数据，按照最小支持度要求，不断生成当前元素的频繁项集。

In [71]:
def mineTree(inTree, headerTable, minSup, preFix, freqItemList):
    """
    创建条件FP树
    
    Parameters
    -----------
        inTree :        myFPtree
        headerTable :   满足minSup的所有元素
        minSup :        最小支持度
        preFix :        preFix为newFreqSet上一次的存储记录
        freqItemList :  用来存储频繁子项的列表
    """
    
    # 满足最小支持度的项集的key
    bigL = [v[0] for v in sorted(headerTable.items(), key=lambda p: p[0])]
#     print('-----', sorted(headerTable.items(), key=lambda p: p[1][0]))
#     print('bigL=', bigL)
    # 循环遍历最频繁的项集的key
    for basePat in bigL:
        newFreqSet = preFix.copy()
        newFreqSet.add(basePat)
#         print('newFreqSet=', newFreqSet, preFix)
        
        freqItemList.append(newFreqSet)
#         print('freqItemList=', freqItemList)
        condPattBases = findPrefixPath(basePat, headerTable[basePat][1])
#         print('condPattBases=', basePat, condPattBases)
        
        # 构建FP树
        myCondTree, myHead = createTree(condPattBases, minSup)
        if myHead != None:
            print('conditional tree for: ', newFreqSet)
            myCondTree.disp()
            print('\n')
            mineTree(myCondTree, myHead, minSup, newFreqSet, freqItemList)

In [72]:
freqItems = []
mineTree(myFPtree, myHeaderTab, 3, set([]), freqItems)

conditional tree for:  {'s'}
     Null Set   1
         x   3


conditional tree for:  {'t'}
     Null Set   1
         z   3
             y   3
                 x   3


conditional tree for:  {'t', 'x'}
     Null Set   1
         y   3
             z   3


conditional tree for:  {'t', 'z', 'x'}
     Null Set   1
         y   3


conditional tree for:  {'t', 'y'}
     Null Set   1
         z   3


conditional tree for:  {'x'}
     Null Set   1
         z   3


conditional tree for:  {'y'}
     Null Set   1
         z   3
             x   3


conditional tree for:  {'y', 'x'}
     Null Set   1
         z   3




In [73]:
freqItems

[{'r'},
 {'s'},
 {'s', 'x'},
 {'t'},
 {'t', 'x'},
 {'t', 'x', 'y'},
 {'t', 'x', 'z'},
 {'t', 'x', 'y', 'z'},
 {'t', 'y'},
 {'t', 'y', 'z'},
 {'t', 'z'},
 {'x'},
 {'x', 'z'},
 {'y'},
 {'x', 'y'},
 {'x', 'y', 'z'},
 {'y', 'z'},
 {'z'}]

## 3. 从Twitter源中发现一些共现词
发现Twitter中的共现词
> 1. 收集数据:使用 Python-twitter 模块来访问推文。
> 2. 准备数据:编写一个函数来去掉URL、去掉标点、转换成小写并从字符串中建立一个单词集合。
> 3. 分析数据:在Python提示符下查看准备好的数据,确保它的正确性。
> 4. 训练算法:使用本章前面开发的 createTree() 与 mineTree() 函数执行FP-growth算法。
> 5. 测试算法:这里不适用。
> 6. 使用算法:本例中没有包含具体应用,可以考虑用于情感分析或者查询推荐领域


无法链接到Twitter。没有进行实验。

## 4. 从新闻网站点击流中挖掘

In [76]:
parsedDat = [line.split() for line in open('kosarak.dat').readlines()]

In [77]:
initSet = createInitSet(parsedDat)
# 构建FP树，寻找至少被10万人浏览过的新闻报道
myFPtree, myHeaderTab = createTree(initSet, 100000)

In [78]:
myFreqList = []
mineTree(myFPtree, myHeaderTab, 100000, set([]), myFreqList)

conditional tree for:  {'1'}
     Null Set   1
         6   132113


conditional tree for:  {'11'}
     Null Set   1
         6   324013
             3   143682
         3   17604


conditional tree for:  {'11', '3'}
     Null Set   1
         6   143682


conditional tree for:  {'3'}
     Null Set   1
         6   265180




In [79]:
len(myFreqList), myFreqList

(9,
 [{'1'},
  {'1', '6'},
  {'11'},
  {'11', '3'},
  {'11', '3', '6'},
  {'11', '6'},
  {'3'},
  {'3', '6'},
  {'6'}])

例如，表明1,6两者被10万人以上浏览过，他们两个具有强相关的联系。